<a href="https://colab.research.google.com/github/ViktorBakken/SM-r/blob/main/models/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
import numpy as np

import sklearn.linear_model as skl_lm
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV

from scipy.stats import uniform


seed = 42

ModuleNotFoundError: No module named 'pandas'

In [ ]:
#Load data
df = pd.read_csv(os.path.abspath("../data/ormalized_labeled_training_data.csv"))

#Select label column
y = df['increase_stock']

#Select training data
X = df[['hour_of_day', 'day_of_week', 'month', 'holiday','weekday','summertime','temp','dew','humidity','precip','snow','snowdepth','windspeed','cloudcover','visibility']]

# #Create new features
# X['day_or_night'] = X['hour_of_day'].apply(lambda x: 1 if 8 <= x < 21 else 0)
# X['normal_day'] = (~((X['summertime'] == 1) | (X['holiday'] == 1) | (X['weekday'] == 0))).astype(int)
# X['cold'] = X['temp'].apply(lambda x: 1 if x <= 8 else 0)


#Remove bad features
X = X.drop(['snow', 'snowdepth', 'holiday', 'visibility', 'precip', 'dew'], axis=1)

X.info()

#Select label column
y = df['increase_stock']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=seed, stratify=y)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   hour_of_day   1600 non-null   float64
 1   day_of_week   1600 non-null   float64
 2   month         1600 non-null   float64
 3   weekday       1600 non-null   float64
 4   summertime    1600 non-null   float64
 5   temp          1600 non-null   float64
 6   humidity      1600 non-null   float64
 7   windspeed     1600 non-null   float64
 8   cloudcover    1600 non-null   float64
 9   day_or_night  1600 non-null   int64  
 10  normal_day    1600 non-null   int64  
 11  cold          1600 non-null   int64  
dtypes: float64(9), int64(3)
memory usage: 150.1 KB


In [ ]:

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', skl_lm.LogisticRegression(max_iter=1000, random_state=seed))
])

# Define hyperparameter search space
param_distributions = {
    'log_reg__C': uniform(0.01, 10),  # C is the inverse of regularization strength
    'log_reg__penalty': ['l2'],
    'log_reg__solver': ['lbfgs', 'saga']
}

# Set up cross-validation with 10 folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=3000,
    scoring='f1',
    refit=True,
    cv=cv,
    n_jobs=-1,
    verbose=3,
    return_train_score=False,
    random_state=seed,
)

# Fit the model using RandomizedSearchCV
random_search.fit(X_train, y_train)

print("Best Parameters:")
print(random_search.best_params_)
print(f"\nBest {random_search.refit} Score: {random_search.best_score_:.4f}")


# Evaluate the best model on the test set
best_model = random_search.best_estimator_


Fitting 10 folds for each of 3000 candidates, totalling 30000 fits
Best Parameters:
{'log_reg__C': 9.74755518841459, 'log_reg__penalty': 'l2', 'log_reg__solver': 'lbfgs'}

Best True Score: 0.4550


In [ ]:
#Prediction
y_pred = best_model.predict(X_test)

#Probability
y_proba = best_model.predict_proba(X_test)[:, 1]


In [ ]:
# Print evaluation metrics
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"Test F1-Weighted: {f1_score(y_test, y_pred, average='weighted'):.4f}")
print(f"Recall Score: {recall_score(y_test, y_pred):.4f}")
print(f"Precision Score: {precision_score(y_test, y_pred):.4f}")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

from sklearn.metrics import classification_report
print("\n"+ classification_report(y_test, y_pred))

F1 Score: 0.5591
Accuracy Score: 0.8719
Recall Score: 0.4483
Precision Score: 0.7429
ROC AUC Score: 0.8891

              precision    recall  f1-score   support

         0.0       0.89      0.97      0.93       262
         1.0       0.74      0.45      0.56        58

    accuracy                           0.87       320
   macro avg       0.82      0.71      0.74       320
weighted avg       0.86      0.87      0.86       320



In [5]:
import os
import pandas as pd
import numpy as np

import sklearn.linear_model as skl_lm
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, train_test_split, RandomizedSearchCV

from scipy.stats import uniform


seed = 42

In [6]:
#Load data
df = pd.read_csv(os.path.abspath("normalized_labeled_training_data.csv"))

#Select label column
y = df['increase_stock']

#Select training data
X = df[['hour_of_day', 'day_of_week', 'month', 'holiday','weekday','summertime','temp','dew','humidity','precip','snow','snowdepth','windspeed','cloudcover','visibility']]

#Create new features
X['day_or_night'] = X['hour_of_day'].apply(lambda x: 1 if 8 <= x < 21 else 0)
X['normal_day'] = (~((X['summertime'] == 1) | (X['holiday'] == 1) | (X['weekday'] == 0))).astype(int)
X['cold'] = X['temp'].apply(lambda x: 1 if x <= 8 else 0)


#Remove bad features
X = X.drop(['snow', 'snowdepth', 'holiday', 'visibility', 'precip', 'dew'], axis=1)

X.info()

#Select label column
y = df['increase_stock']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=seed, stratify=y)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600 entries, 0 to 1599
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   hour_of_day   1600 non-null   float64
 1   day_of_week   1600 non-null   float64
 2   month         1600 non-null   float64
 3   weekday       1600 non-null   float64
 4   summertime    1600 non-null   float64
 5   temp          1600 non-null   float64
 6   humidity      1600 non-null   float64
 7   windspeed     1600 non-null   float64
 8   cloudcover    1600 non-null   float64
 9   day_or_night  1600 non-null   int64  
 10  normal_day    1600 non-null   int64  
 11  cold          1600 non-null   int64  
dtypes: float64(9), int64(3)
memory usage: 150.1 KB


In [ ]:

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', skl_lm.LogisticRegression(max_iter=1000, random_state=seed))
])

# Define hyperparameter search space
param_distributions = {
    'log_reg__C': uniform(0.01, 10),  # C is the regularization strength
    'log_reg__penalty': ['l2'],
    'log_reg__solver': ['lbfgs', 'saga']
}

# Set up cross-validation with 10 folds
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

# RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=param_distributions,
    n_iter=300,
    scoring='f1',
    refit=True,
    cv=cv,
    n_jobs=-1,
    verbose=3,
    return_train_score=False,
    random_state=seed,
)

# Fit the model using RandomizedSearchCV
random_search.fit(X_train, y_train)

print("Best Parameters:")
print(random_search.best_params_)
print(f"\nBest {random_search.refit} Score: {random_search.best_score_:.4f}")


# Evaluate the best model on the test set
best_model = random_search.best_estimator_


Fitting 10 folds for each of 300 candidates, totalling 3000 fits
Best Parameters:
{'log_reg__C': 9.74755518841459, 'log_reg__penalty': 'l2', 'log_reg__solver': 'lbfgs'}

Best True Score: 0.4550


In [8]:
#Prediction
y_pred = best_model.predict(X_test)

#Probability
y_proba = best_model.predict_proba(X_test)[:, 1]


In [9]:
# Print evaluation metrics
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"Test F1-Weighted: {f1_score(y_test, y_pred, average='weighted'):.4f}")
print(f"Recall Score: {recall_score(y_test, y_pred):.4f}")
print(f"Precision Score: {precision_score(y_test, y_pred):.4f}")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

from sklearn.metrics import classification_report
print("\n"+ classification_report(y_test, y_pred))

F1 Score: 0.5591
Test F1-Weighted: 0.8587
Recall Score: 0.4483
Precision Score: 0.7429
ROC AUC Score: 0.8891

              precision    recall  f1-score   support

         0.0       0.89      0.97      0.93       262
         1.0       0.74      0.45      0.56        58

    accuracy                           0.87       320
   macro avg       0.82      0.71      0.74       320
weighted avg       0.86      0.87      0.86       320

